In [39]:
!pip install praw python-dotenv pandas --quiet
!pip install sqlalchemy
!pip install psycopg2-binary


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [45]:
import os
import praw
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

In [46]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
## normally have .env writer cell here

In [48]:
import pandas as pd

# Save to CSV
df_reddit = pd.DataFrame(posts)
df_reddit.to_csv("reddit_snap_sentiment_raw.csv", index=False)

In [ ]:
## round  2... fight ! 
import os
import praw
from datetime import datetime
import pandas as pd

# Authenticate Reddit
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
    username=os.getenv("REDDIT_USERNAME"),
    password=os.getenv("REDDIT_PASSWORD"),
    check_for_async=False
)

# Define subreddits and search terms
subreddits = [
    "cscareerquestions", "jobs", "overemployed", "experienceddevs", 
    "techcareers", "recruitinghell", "bigtechsalaries", "WorkReform",
    "csMajors", "fired", "techlayoffs", "internships"
]

search_terms = [
    "snap inc", "snapchat", "working at snap", "snap layoffs",
    "snapchat layoffs", "snapchat engineer", "snapchat interview",
    "snapchat recruiter", "snapchat compensation", "snapchat return to office",
    "snapchat remote work", "snapchat base salary", "snapchat culture",
    "snapchat offer rescinded", "snapchat firing", "snapchat pay"
]

# Start pulling posts
posts = []

for subreddit in subreddits:
    for term in search_terms:
        print(f" Searching r/{subreddit} for '{term}'...")
        try:
            for post in reddit.subreddit(subreddit).search(term, sort="relevance", limit=100):
                if post.score >= 2 and post.selftext and post.selftext.strip():
                    posts.append({
                        "subreddit": subreddit,
                        "id": post.id,
                        "title": post.title,
                        "author": str(post.author),
                        "score": post.score,
                        "num_comments": post.num_comments,
                        "created_utc": datetime.fromtimestamp(post.created_utc),
                        "selftext": post.selftext,
                        "permalink": f"https://www.reddit.com{post.permalink}"
                    })
        except Exception as e:
            print(f" Error in r/{subreddit} with term '{term}':", e)

# Create DataFrame and clean
df_reddit = pd.DataFrame(posts).drop_duplicates(subset="id")
df_reddit["created_utc"] = pd.to_datetime(df_reddit["created_utc"])
df_reddit["created_month"] = df_reddit["created_utc"].dt.to_period("M")
df_reddit["year"] = df_reddit["created_utc"].dt.year
df_reddit["combined_text"] = (df_reddit["title"].fillna("") + " " + df_reddit["selftext"].fillna("")).str.lower()

# Tag each post with relevant themes
def tag_themes(text):
    tags = []
    if any(k in text for k in ["layoff", "fired", "return to office", "rto", "remote only"]):
        tags.append("layoffs_rto")
    if any(k in text for k in ["culture", "burnout", "toxic", "execs", "management"]):
        tags.append("culture_fit")
    if any(k in text for k in ["ml", "machine learning", "ai", "generative"]):
        tags.append("ml_ai_roles")
    if any(k in text for k in ["salary", "pay", "rsu", "stock", "compensation"]):
        tags.append("compensation")
    if any(k in text for k in ["interview", "hackerrank", "onsite", "offer rescinded"]):
        tags.append("interview_process")
    return tags

df_reddit["relevance_tags"] = df_reddit["combined_text"].apply(tag_themes)
df_reddit["era"] = df_reddit["created_utc"].apply(
    lambda x: "recent" if x >= pd.Timestamp("2023-01-01") else "older"
)

#  filter to only keep posts with at least one relevant tag
df_reddit = df_reddit[df_reddit["relevance_tags"].apply(len) > 0]

print(f"\n Total tagged posts collected: {len(df_reddit)}")
df_reddit.head(5)

🔍 Searching r/cscareerquestions for 'snap inc'...
🔍 Searching r/cscareerquestions for 'snapchat'...
🔍 Searching r/cscareerquestions for 'working at snap'...
🔍 Searching r/cscareerquestions for 'snap layoffs'...
🔍 Searching r/cscareerquestions for 'snapchat layoffs'...
🔍 Searching r/cscareerquestions for 'snapchat engineer'...
🔍 Searching r/cscareerquestions for 'snapchat interview'...
🔍 Searching r/cscareerquestions for 'snapchat recruiter'...
🔍 Searching r/cscareerquestions for 'snapchat compensation'...
🔍 Searching r/cscareerquestions for 'snapchat return to office'...
🔍 Searching r/cscareerquestions for 'snapchat remote work'...
🔍 Searching r/cscareerquestions for 'snapchat base salary'...
🔍 Searching r/cscareerquestions for 'snapchat culture'...
🔍 Searching r/cscareerquestions for 'snapchat offer rescinded'...
🔍 Searching r/cscareerquestions for 'snapchat firing'...
🔍 Searching r/cscareerquestions for 'snapchat pay'...
🔍 Searching r/jobs for 'snap inc'...
🔍 Searching r/jobs for 'sn

,subreddit,id,title,author,score,num_comments,created_utc,selftext,permalink,created_month,year,combined_text,relevance_tags,era
0,cscareerquestions,z816qv,Snap requiring employees to work from office 8...,gpacsu,824,366,2022-11-29 18:03:39,https://www.reuters.com/technology/snap-employ...,https://www.reddit.com/r/cscareerquestions/com...,2022-11,2022,snap requiring employees to work from office 8...,"[layoffs_rto, ml_ai_roles]",older
1,cscareerquestions,57l04j,Snap Inc vs. Two Sigma vs. Airbnb,NewHorizons18,22,35,2016-10-15 06:59:53,I have offers from all of the above companies ...,https://www.reddit.com/r/cscareerquestions/com...,2016-10,2016,snap inc vs. two sigma vs. airbnb i have offer...,[ml_ai_roles],older
2,cscareerquestions,581s3x,Snap Inc. Stock,whichtochooz,54,28,2016-10-18 03:40:40,Hello. I have incredible offers from Snap Inc....,https://www.reddit.com/r/cscareerquestions/com...,2016-10,2016,snap inc. stock hello. i have incredible offer...,"[ml_ai_roles, compensation]",older
3,cscareerquestions,5aslhk,Snap Inc. (Snapchat) HackerRank challenge,princemaxx,24,29,2016-11-02 21:12:52,Anyone done a HackerRank for Snap Intern? \nIs...,https://www.reddit.com/r/cscareerquestions/com...,2016-11,2016,snap inc. (snapchat) hackerrank challenge anyo...,[interview_process],older
6,cscareerquestions,6vgr9u,Snap Inc. New Grad interview,smarirs,4,6,2017-08-23 04:54:28,Has anyone interviewed with Snap Inc. for a ne...,https://www.reddit.com/r/cscareerquestions/com...,2017-08,2017,snap inc. new grad interview has anyone interv...,[interview_process],older


In [55]:
# Save cleaned and tagged Reddit data to CSV
df_reddit.to_csv("reddit_snap_cleaned_tagged.csv", index=False)
print("📁 Exported to 'reddit_snap_cleaned_tagged.csv'")

📁 Exported to 'reddit_snap_cleaned_tagged.csv'


In [54]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [ ]:
from sqlalchemy import create_engine
import os
import pandas as pd

# Get DB credentials
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Drop the table with CASCADE manually (compatible version)
raw_conn = engine.raw_connection()
cur = raw_conn.cursor()
try:
    cur.execute("DROP TABLE IF EXISTS reddit_posts CASCADE;")
    raw_conn.commit()
    print(" Dropped existing reddit_posts table with CASCADE.")
except Exception as e:
    raw_conn.rollback()
    print("❌ Drop failed:", e)
finally:
    cur.close()
    raw_conn.close()

# Convert problematic types
df_reddit["created_month"] = df_reddit["created_month"].astype(str)

# Push data
df_reddit.to_sql("reddit_posts", con=engine, if_exists="replace", index=False)
print(" Successfully pushed cleaned reddit_posts to Postgres.")


✅ Dropped existing reddit_posts table with CASCADE.
✅ Successfully pushed cleaned reddit_posts to Postgres.
